In [1]:
import ipywidgets as widgets
from IPython.display import display

from gridlab import Action, ViewPipeline, World, create_world, verify_all_solutions, world_names
from gridlab.view import HTMLTableLegendView, HTMLTableStatusView, HTMLGridView

In [2]:
STYLE  = """
.legend tr, .status tr, .grid tr {
    background-color: #ffffff;
}

.legend td, .status  td {
   border: 1px solid #000000;
}

.legend td.entity {
    font-family: 'Source Code Pro', monospace;
    text-align: center;
}

.status td.status-key {
    font-weight: bold;
}

.grid td {
    font-family: 'Source Code Pro', monospace;
    font-size: 24px;
    text-align: center;
    width: 1.5em;
    height: 1.5em;
}

.grid span {
    font-family: 'Source Code Pro', monospace;
    font-size: 24px;
    line-height: 1em;
}
"""

In [3]:
class GridWorldWidget:
    def __init__(self, world: World, pipeline: ViewPipeline):
        self.world = world
        self.pipeline = pipeline
        self.views = pipeline.render(world)

        # Widgets for display
        self.legend_display = widgets.HTML(self._format_legend())
        self.status_display = widgets.HTML(self._format_status())
        self.grid_display = widgets.HTML(self._format_grid())

        # Control buttons
        self.up_button = widgets.Button(description='⏶', layout=widgets.Layout(width='40px'))
        self.down_button = widgets.Button(description='⏷', layout=widgets.Layout(width='40px'))
        self.left_button = widgets.Button(description='⏴', layout=widgets.Layout(width='40px'))
        self.right_button = widgets.Button(description='⏵', layout=widgets.Layout(width='40px'))

        # Button click callbacks
        self.up_button.on_click(lambda _: self._on_move(Action.UP))
        self.down_button.on_click(lambda _: self._on_move(Action.DOWN))
        self.left_button.on_click(lambda _: self._on_move(Action.LEFT))
        self.right_button.on_click(lambda _: self._on_move(Action.RIGHT))

        # Layout
        controls_top = widgets.HBox(
            [self.up_button],
            layout=widgets.Layout(
                align_items='center',
                justify_content='center',
            ),
        )
        controls_bottom = widgets.HBox(
            [self.left_button, self.down_button, self.right_button],
            layout=widgets.Layout(
                align_items='center',
                justify_content='center',
            ),
        )
        controls = widgets.VBox(
            [controls_top, controls_bottom],
            layout=widgets.Layout(padding='20px'),
        )

        page = widgets.VBox(
            [
                self.legend_display,
                self.status_display,
                self.grid_display,
                controls,
            ],
        )

        wrapper = widgets.HBox([page], layout=widgets.Layout(padding='20px'))
        display(wrapper)

    def _format_legend(self):
        theme_style = self.pipeline.theme.css()
        style = f'{STYLE}\n\n{theme_style}'
        legend = self.views['legend']
        return f'<style>{style}</style><div><b>Legend</b>{legend}</div>'

    def _format_status(self):
        status = self.views['status']
        return f'<div><b>Status</b>{status}</div>'

    def _format_grid(self):
        grid = self.views['grid']
        return f'<div><b>Grid</b>{grid}</div>'

    def _on_move(self, action: Action):
        self.world.step(action=action)
        self.views = self.pipeline.render(self.world)

        # Refresh displays in-place
        self.legend_display.value = self._format_legend()
        self.status_display.value = self._format_status()
        self.grid_display.value = self._format_grid()

In [4]:
def list_worlds(verified: bool = True):
    if not verified:
        names = world_names()
    else:
        status = verify_all_solutions(report=False)
        names = list(status['success'].keys())

    print(*(f'- {name}' for name in names), sep='\n')


list_worlds()

- empty
- demo
- causeway
- fog
- spike
- timer
- mirror
- mirror-block
- mirror-block-flip
- patrol
- chase


In [5]:
def run(name: str = 'causeway'):
    world = create_world(name)
    pipeline = ViewPipeline(
        {
            'legend': HTMLTableLegendView(),
            'status': HTMLTableStatusView(),
            'grid': HTMLGridView(),
        },
        theme='desert',
    )
    _ = GridWorldWidget(world, pipeline=pipeline)

run()